# Generate demand profiles for RC dwellings

This notebook reproduces the standalone script in `generate_demand_profiles.py` and can be used interactively to compute the hybrid heat-pump dispatch for every dwelling contained in `Codes/Data/1R1C1P1S_filtered.csv` using the EOH2303 weather data for 9–13 February 2022.

In [ ]:
from pathlib import Path
import pandas as pd

from generate_demand_profiles import (
    DATA_DIR,
    OUTPUT_DIR,
    START_DATE,
    END_DATE,
    load_dwelling_parameters,
    prepare_weather_profile,
    run_dispatch_for_dwelling,
    build_tariff,
)

## Load dwelling parameters

The summary CSV contains the R, C, and gain coefficients for each dwelling. We extract the unique dwelling identifiers and store them for later use.

In [ ]:
params_map = load_dwelling_parameters(DATA_DIR / '1R1C1P1S_filtered.csv')
print(f"Loaded {len(params_map)} dwellings from the summary file.")
# Display the first entry
next(iter(params_map.items()))

## Prepare the weather profile

Load the merged EOH2303 weather data, align it to the requested period, and clean any missing values.

In [ ]:
weather = prepare_weather_profile(DATA_DIR / 'EOH2303_merged_30min.csv', START_DATE, END_DATE)
weather.head()

## Build the tariff profile

The optimization requires the day-ahead tariff. We reuse the helper from the optimization package to generate the half-hourly tariff for the requested window.

In [ ]:
n_days = (END_DATE.normalize() - START_DATE.normalize()).days + 1
tariff = build_tariff(START_DATE.normalize(), n_days=n_days, step='30min')
tariff = tariff.reindex(weather.index)
if tariff.isna().any().any():
    raise ValueError('Tariff contains NaN values after alignment; adjust the requested range.')
tariff.head()

## Run the optimization per dwelling

Iterate through each dwelling, run the hybrid heat-pump optimization, and collect the results into a single `dict` keyed by dwelling identifier. This may take several minutes depending on the solver setup and number of dwellings.

In [ ]:
all_results = {}
for dwelling_id, params in params_map.items():
    result = run_dispatch_for_dwelling(params, weather, tariff)
    all_results[dwelling_id] = result

combined = pd.concat(all_results, axis=1)
combined.head()

## Save the combined demand profiles

Finally, persist the concatenated dispatch outputs to the `Codes/Output` directory. The CSV contains one group of columns per dwelling, matching the output of the script.

In [ ]:
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
output_path = OUTPUT_DIR / 'demand_profiles_EOH2303_Feb09-13.csv'
combined.to_csv(output_path, index_label='timestamp')
print(f'Saved demand profiles for {len(all_results)} dwellings to {output_path}')